In [1]:
import os,pickle
import numpy as np
import tensorflow as tf
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt

from process_data import preprocess_conditional_flow_data_cww
from Model.ConditionalRealNVP import ConditionalRealNVP
from Utils.ObjDict import ObjDict
from Utils.mkdir_p import mkdir_p

In [2]:
# __________________________________________________________________ ||
# Basic configurables
# __________________________________________________________________ ||

input_csv_path = "data/train_cww.npy"
saved_model_path = "output/train_condrealnvp_cww_210208_v1/saved_model_700.h5"
output_dir = os.path.dirname(saved_model_path)
event_size = 4000
ndim = 3
ncond = 1

NameError: name 'os' is not defined

In [12]:
# __________________________________________________________________ ||
# Load models
# __________________________________________________________________ ||

event_size = 1
nf_model = ConditionalRealNVP(num_coupling_layers=10,ndim=ndim,ncond=ncond)
samples = nf_model.distribution.sample(event_size)
condition = 1.0 * np.ones((event_size,1))
_,_ = nf_model.predict([samples,condition,])
nf_model.load_weights(saved_model_path)

In [6]:
arr = np.load(input_csv_path)
arr_list,_ = preprocess_conditional_flow_data_cww(arr)

In [26]:
# __________________________________________________________________ ||
# Make plots for different conditions
# __________________________________________________________________ ||

n_dim = 3
param_grid = [arr_list[idx_param] for idx_param in np.random.randint(0,len(arr_list),n_dim*n_dim)]
param_grid.sort(key=lambda x: x.condition[0])
figsize = (50,50)
event_size = 5000

samples = nf_model.distribution.sample(event_size)
fig_m4l,ax_m4l = plt.subplots(n_dim,n_dim,figsize=figsize)
fig_mz1,ax_mz1 = plt.subplots(n_dim,n_dim,figsize=figsize)
fig_mz2,ax_mz2 = plt.subplots(n_dim,n_dim,figsize=figsize)

for i,m in enumerate(param_grid):
    
    ix = int(i / n_dim)
    iy = i % n_dim
    
    condition_str = str(m.condition[0])
    condition = np.ones((event_size,1)) * m.condition[0]

    idx_batch = np.random.randint(0,m.x.shape[0],event_size)
    
    x_true = m.x[idx_batch]
    x_gen,_ = nf_model.predict([samples,condition,])

    ax_m4l[ix,iy].hist(x_true[:,0],bins=100,density=1.,histtype='step',range=[-10.,10.],label='True '+condition_str)
    ax_m4l[ix,iy].hist(x_gen[:,0],bins=100,density=1.,histtype='step',range=[-10.,10.],label='Flow '+condition_str)
    ax_m4l[ix,iy].legend(loc='best')
    ax_m4l[ix,iy].set_title(condition_str)
    
    ax_mz1[ix,iy].hist(x_true[:,1],bins=100,density=1.,histtype='step',range=[-10.,10.],label='True '+condition_str)
    ax_mz1[ix,iy].hist(x_gen[:,1],bins=100,density=1.,histtype='step',range=[-10.,10.],label='Flow '+condition_str)
    ax_mz1[ix,iy].legend(loc='best')
    ax_mz1[ix,iy].set_title(condition_str)
    
    ax_mz2[ix,iy].hist(x_true[:,2],bins=100,density=1.,histtype='step',range=[-10.,10.],label='True '+condition_str)
    ax_mz2[ix,iy].hist(x_gen[:,2],bins=100,density=1.,histtype='step',range=[-5.,5.],label='Flow '+condition_str)
    ax_mz2[ix,iy].legend(loc='best')
    ax_mz2[ix,iy].set_title(condition_str)
    
fig_m4l.savefig(os.path.join(output_dir,'m4l.png'))
fig_mz1.savefig(os.path.join(output_dir,'mZ1.png'))
fig_mz2.savefig(os.path.join(output_dir,'mZ2.png'))

/Users/lucien/.pyenv/versions/3.7.6/lib/python3.7/site-packages/ipykernel/__main__.py:13: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).


In [3]:
# __________________________________________________________________ ||
# Make plots for likelihood
# __________________________________________________________________ ||

import time

n_dim = 5
select_param = [5,30,45,60,80]
param_grid = [arr_list[idx_param] for idx_param in select_param]
param_grid.sort(key=lambda x: x.condition[0])
figsize = (25,10)

plot_low = 0.0
plot_high = 0.2
n_grid = 100
x_grid = [plot_low+(plot_high-plot_low)/n_grid*i for i in range(n_grid+1)]

fig, ax = plt.subplots(len(select_param),n_dim,figsize=figsize,constrained_layout=True)

def make_array(event_size,x_grid,p,nf_model):
    log_prob = np.zeros(n_grid+1)
    log_prob_sm = np.zeros(n_grid+1)
    
    idx_batch = np.random.randint(0,p.x.shape[0],event_size)
    condition_concat = np.concatenate([np.ones((event_size,1)) * x for ix,x in enumerate(x_grid)])
    x_data_concat = np.concatenate([p.x[idx_batch] for ix,x in enumerate(x_grid)])
    
    z_concat = nf_model.batch_log_loss([x_data_concat,condition_concat])

    for ig,x in enumerate(x_grid):
        arg = tf.expand_dims(z_concat[ig*idx_batch.shape[0]:(ig+1)*idx_batch.shape[0]],axis=1)
        log_prob[ig] = tf.reduce_sum(nf_model.distribution.log_prob(arg))
    return log_prob.astype(np.float64),idx_batch

for i,p in enumerate(param_grid):
    
    print("-"*100)
    print("Drawing plot ",i," with param ",p.condition[0])
    
    #ix = int(i / n_dim)
    #iy = i % n_dim
    ix = i
    
    start_time = time.time()
    
    condition_str = str(p)
    
    for event_size in [5,50,1000]:
        nf_model.direction = -1
        log_prob,idx_batch = make_array(event_size,x_grid,p,nf_model)
        
        y_grid = tf.nn.softmax(log_prob/event_size) * n_grid
        ax[ix,0].plot(x_grid,y_grid,label=str(event_size)+' events',)
        ax[ix,0].set_title("cww: "+"%4.4f"%p.param)
        ax[ix,0].legend(loc='best')
        ax[ix,0].grid(True)
        ax[ix,0].set_ylim(0.,40.)
        ylims = ax[ix,0].get_ylim()
        ax[ix,0].arrow(p.condition[0], ylims[1], 0., ylims[0]-ylims[1],)
        
        ax[ix,1].plot(x_grid,-log_prob-np.min(-log_prob),label=str(event_size)+' events',)
        ax[ix,1].set_title("cww: "+"%4.4f"%p.param)
        ax[ix,1].legend(loc='best')
        ax[ix,1].grid(True)
        ax[ix,1].set_ylim(0.,10.)
        ylims = ax[ix,1].get_ylim()
        ax[ix,1].arrow(p.condition[0], ylims[1], 0., ylims[0]-ylims[1],)
        
        if event_size == 1000:
            nf_model.direction = 1
            samples = nf_model.distribution.sample(idx_batch.shape[0])
            x_pred,_ = nf_model.predict([samples,np.ones((idx_batch.shape[0],1))*p.param])
            ax[ix,2].hist(p.x[idx_batch,0],bins=50,histtype='step',range=[-1.,1.],label='True',)
            ax[ix,2].hist(x_pred[:,0],bins=50,histtype='step',range=[-1.,1.],label='Flow',)
            ax[ix,2].legend(loc='best')
            
            ax[ix,3].hist(p.x[idx_batch,1],bins=20,histtype='step',range=[-2.,2.],label='True',)
            ax[ix,3].hist (x_pred[:,1],bins=20,histtype='step',range=[-2.,2.],label='Flow',)
            ax[ix,3].legend(loc='best')
            
            ax[ix,4].hist(p.x[idx_batch,2],bins=20,histtype='step',range=[0.,5.],label='True',)
            ax[ix,4].hist(x_pred[:,2],bins=20,histtype='step',range=[0.,5.],label='Flow',)
            ax[ix,4].legend(loc='best')
    
    elapsed_time = time.time() - start_time
    print("Time used: "+str(elapsed_time)+"s")
    
fig.savefig(os.path.join(output_dir,'log_loss.png'))

NameError: name 'arr_list' is not defined